In [1]:
import yaml
import src.testbed_performance as testbed_performance
import src.retrieve_logs as retrieve_logs
import src.twingate_status as twingate_status
from datetime import datetime
import os 
import yaml
import pathlib
import src.yaml_loader as yaml_loader

Twingate is running.


In [ ]:
benchmark_session_ids = []

In [9]:
jenkins_url = "http://jenkins-ber.reeinfra.net/view/Main%20Testbed%20Queues/"
job_name = "job/release-testbed-validation-gamma"
build_info_url = f"{jenkins_url}{job_name}"
branch = "rc59.0.6"

In [3]:
current_dir = os.getcwd()
credentials = yaml_loader.load_yaml(pathlib.Path(f"{current_dir}/../credentials/credentials.yaml").open().read())
last_successful_build = retrieve_logs.get_last_build_information("Successful", credentials, build_info_url)


In [4]:
last_successful_build

3497

In [10]:
if branch:
    all_builds_for_dedicated_branch = testbed_performance.get_all_branch_builds(branch, last_successful_build, credentials, build_info_url)
    print("\n".join([str(x) for x in all_builds_for_dedicated_branch]))

Error: Couldn't retrieve any information.


KeyError: 'branch'

In [11]:
retrieve_logs.get_last_build_information("Successful", credentials, build_info_url)

3497

In [18]:
auth=(credentials["username"], credentials["credentials"])

In [19]:
retrieve_logs.get_session_information(credentials, 3497, build_info_url)

TypeError: 'int' object is not subscriptable

In [ ]:
import requests
jenkins_url = "http://jenkins-ber.reeinfra.net/view/Main%20Testbed%20Queues/"
job_name = "job/release-testbed-validation-gamma"
build_info_url = f"{jenkins_url}{job_name}"

console_log = requests.get(f"{build_info_url}/3497/api/json",
            auth=(credentials["username"], credentials["credentials"]))   
result = console_log.json()["result"]

In [8]:
if not twingate_status.is_twingate_running():
    print("Twingate is not running. Please start it and try again.")
    exit(1)

credentials = yaml_loader.load_yaml(pathlib.Path(f"../credentials/credentials.yaml").open().read()
    )
last_successful_build = retrieve_logs.get_last_build_information("Successful", credentials, build_info_url)
console_log = retrieve_logs.get_console_logs(credentials, last_successful_build, build_info_url)
list_of_builds = testbed_performance.get_last_3_succesfull_builds(last_successful_build, credentials, build_info_url)
metrics_yaml = testbed_performance.get_log_yaml(console_log)

#retrieve metrics and calculate averag
all_metrics = testbed_performance.collect_metrics(list_of_builds, credentials, build_info_url)
metrics_average = testbed_performance.get_metrics_average(all_metrics)

Error: Couldn't retrieve any information.


KeyError: 'branch'

In [ ]:
front_camera_packets = 'front camera packets per second_packets_per_second'

list_of_keys_camera_bitrate = ['front camera bitrate_kbits_per_second', 
                'left camera bitrate_kbits_per_second',
                'right camera bitrate_kbits_per_second',
                'back camera bitrate_kbits_per_second']

list_of_keys_camera_new_unit = ['front camera Mbit/s',
                              'left camera Mbit/s',
                              'right camera Mbit/s',
                              'back camera Mbit/s']

list_of_keys_udp_tcp_bitrate = ['ROS UDP_kbits_per_second',
                'ROS TCP_kbits_per_second',
                'UDP TS-VE_kbits_per_second',
                'UDP VE-TS_kbits_per_second',
                'TCP TS-VE_kbits_per_second',
                'TCP VE-TS_kbits_per_second']

list_of_keys_udp_tcp_new_unit = ['ROS UDP Mbit/s', 
                         'ROS TCP Mbit/s',
                         'UDP TS-VE Mbit/s',
                         'UDP VE-TS Mbit/s',
                         'TCP TS-VE Mbit/s',
                         'TCP VE-TS Mbit/s']
print('Average of following builds:', [build['build'] for build in list_of_builds], "on Branch:", list_of_builds[0]["branch"])
print('front camera packets/s','\t\t',
      round(metrics_average[front_camera_packets]['minimum']['value']), 
      round(metrics_average[front_camera_packets]['median']['value']),
      round(metrics_average[front_camera_packets]['max']['value']))

for key in zip(list_of_keys_camera_bitrate, list_of_keys_camera_new_unit):
      if len(key[1]) < 16:
            tab = '\t\t\t'
      else:
            tab = '\t\t'
      print(key[1],tab,
          round(metrics_average[key[0]]['minimum']['value']/1000,2), 
          round(metrics_average[key[0]]['median']['value']/1000,2),
          round(metrics_average[key[0]]['max']['value']/1000,2))
print("----------------------------------------")
print("UDP/TCP Bitrate")
for key in zip(list_of_keys_udp_tcp_bitrate, list_of_keys_udp_tcp_new_unit):

      if len(key[1]) < 16:
            tab = '\t\t\t'
      else:
            tab = '\t\t'

      print(key[1],tab,
          round(metrics_average[key[0]]['minimum']['value']/1000,2), 
          round(metrics_average[key[0]]['median']['value']/1000,2),
          round(metrics_average[key[0]]['max']['value']/1000,2))

### ________________________________________________
### Cells below are outdated as we don't use redash anymore

* Following cells generate a links to redash dashboards for the testbed runs

In [ ]:
# print("test bed session IDs for redash:")
# print("----------------------------------------")
# for build in list_of_builds:
#     print(build['session_id'])

In [ ]:

# p_session_id = [0,0,0,0,0,0,0,0]

# for i in range(min(len(list_of_builds), len(p_session_id))):
#     p_session_id[i] = list_of_builds[i]['session_id']

# start_date = datetime.fromtimestamp(list_of_builds[0]['time']//1000).strftime('%Y-%m-%d')
# end_date = datetime.fromtimestamp(list_of_builds[-1]['time']//1000).strftime('%Y-%m-%d')
# release_candidate = list_of_builds[0]['branch']


# print("Testbed: release testing metrics - v2")
# print(f"https://redash.reeinfra.net/dashboards/96-testbed-release-testing-metrics---v2?p_date={start_date}--{end_date}&p_date.end={end_date}&p_date.start={start_date}&p_date_1=0&p_date_2=0&p_env=prod&p_session_id_1={p_session_id[0]}&p_session_id_2={p_session_id[1]}&p_session_id_3={p_session_id[2]}&p_session_id_4={p_session_id[3]}&p_session_id_5={p_session_id[4]}&p_session_id_6={p_session_id[5]}&p_session_id_7={p_session_id[6]}&p_session_id_8={p_session_id[7]}&p_subsample_percent=5&p_vdrive_version={release_candidate}&refresh=600")

In [ ]:
# p_session_id = [0,0,0,0,0,0,0,0]

# for i in range(min(len(benchmark_session_ids = []), len(p_session_id))):
#     p_session_id[i] = benchmark_session_ids[i]
# print("Release Testing / Production Metrics")
# print(f"https://redash.reeinfra.net/dashboards/61-release-testing-production-metrics?p_env=prod&p_session_id_1={p_session_id[0]}&p_session_id_2={p_session_id[1]}&p_session_id_3={p_session_id[2]}&p_session_id_4={p_session_id[3]}&p_session_id_5={p_session_id[4]}&p_session_id_6={p_session_id[5]}&p_session_id_7={p_session_id[6]}&p_session_id_8={p_session_id[7]}&p_w1652_subsample_percent=5&p_w1653_subsample_percent=5&p_w1656_subsample_percent=5")